In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append('/data/home/jdavey/notebooks/fastai/mi')
from data.loader import *
from nn.lin import *
from nn.relu import *
from nn.softmaxCrossEntropy import *

In [3]:
import math
import torch

## data_loader

In [4]:
avaliable_urls()

MNIST_URL http://deeplearning.net/data/mnist/mnist.pkl


In [5]:
def normalize(x, m, s): return (x-m)/s
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [7]:
# num hidden
nh = 50

In [8]:
# simplified kaiming init / he init
#784, 50
w1 = torch.randn(m,nh)*math.sqrt(2./m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,10)*math.sqrt(2./nh)
b2 = torch.zeros(10)

In [9]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = SoftmaxCrossEntropy()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def forward(self, x):
        for l in self.layers: x = l(x)
        return self.loss.softmax_forward(x)
    
    def backward(self, y_train):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()
    
    def accuracy(self, pred, yb): return (torch.argmax(pred, dim=1)==yb).float().mean()

w1.g,b1.g,w2.g,b2.g = [None]*4

In [11]:
model = Model()
print(model.layers)
loss = model(x_train, y_train); loss

[<nn.lin.Lin object at 0x7f0dd40e1eb8>, <nn.relu.Relu object at 0x7f0dd40e1e80>, <nn.lin.Lin object at 0x7f0dd40e1908>]


tensor(-0.0887)

In [12]:
model.backward(y_train)

In [13]:
pred = model.forward(x_train)
model.accuracy(pred, y_train)

tensor(0.0821)

In [14]:
bs = 10000    # batch size
lr = 0.01   # learning rate
epochs = 20# how many epochs to train for

In [15]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        loss = model(xb, yb)
        model.backward(yb)
        
        parameters = [w1,b1,w2,b2]
        for parameter in parameters:
            parameter-=lr*parameter.g
            
    pred = model.forward(x_train)
    print(model.accuracy(pred, y_train))

tensor(0.1690)
tensor(0.3341)
tensor(0.4370)
tensor(0.5119)
tensor(0.5735)
tensor(0.6174)
tensor(0.6490)
tensor(0.6732)
tensor(0.6910)
tensor(0.7045)
tensor(0.7146)
tensor(0.7235)
tensor(0.7312)
tensor(0.7374)
tensor(0.7428)
tensor(0.7474)
tensor(0.7508)
tensor(0.7551)
tensor(0.7592)
tensor(0.7641)
